# Edward's Bowl of Spaghetti Code

## Imports and DataFrame Initialization

In [31]:
# TODO KETE SETSUNASA NI WA
import pandas as pd
# TODO have input prompt instead of hard-coding
df_OLD = pd.read_excel("files/monthly_production_05-2019.xlsx")
df_NEW = pd.read_excel("files/mothpro2019.xlsx")

#dfRevTest = pd.read_excel('files/monthly_revenue_04-2019.xlsx').fillna(0)
dfCYPro = pd.read_excel('files/federal_production_CY08-17.xlsx')

## For Setting Header Format

In [11]:
# Reads Header Config File
def read_hconfig():
    columns = []
    with open('config/headerdef.txt') as hdef:
        # Seperated by commas, no whitespace plz :(
        columns = hdef.readline().split(',')
    return columns

# Returns Header List based on Excel file
# Use if large amount of Field Names
def set_header(samplefile):
    return list(samplefile.columns)


header = set_header(df_OLD)
print(header)

['Month', 'Calendar Year', 'Land Class', 'Land Category', 'Commodity', 'Volume']


## For Setting Unit Dictionary

In [81]:
# Reads Unit Config File
# Commodity and Unit seperated by an equals sign " = "
# Fast, Best for small number of Units / after initial read
def read_uconfig():
    units = {}
    with open('config/unitdef.txt') as udef:
        for line in udef:
            split = line.split(" = ")
            add_item(split[0], split[1].strip(), units)
    return units


# Returns Unit Dictionary on Excel file
# Very Slow, but better than manually setting large # of units
def make_unit_dict(file):
    units = {}
    for row in file['Product']:
        # For general purpose commodities
        if "(" in row:
            split = row.rsplit(" (",1)
            add_item(split[0], split[1].rstrip(")"), units)
        # The comma is for Geothermal
        elif "," in row:
            split = row.split(", ",1)
            add_item(split[0], split[1],units)
        # In case no unit is found
        else:
            add_item(row, '', units)
            
    return units


# Adds Key with a Set of Values
def add_item(key, value, dictionary):
    if key in dictionary:
        dictionary[key].add(value)
    else:
        dictionary[key] = {value}

        
# Wrties to config to speed up process later
# Maybe have it write seperate files for each type?
def write_units(units):
    config = open("config/unitdef.txt","w")
    for k,v in units.items():
        for u in v:
            line = k + ' = ' +  u.strip("'")  + '\n'
            config.write(line)
        
        
write_units(make_unit_dict(dfCYPro))
units = read_uconfig()
units

{'Gas': {'mcf'},
 'Oil': {'bbl'},
 'Coal': {'tons'},
 'Wavellite': {'spcmn'},
 'Quartz Crystal': {'tickets/pounds'},
 'Salt': {'tons'},
 'Geothermal - Electrical Generation': {'Kilowatt Hours',
  'Other',
  'Thousands of Pounds'},
 'Geothermal - Direct Utilization': {'Hundreds of Gallons',
  'Millions of BTUs'},
 'Anhydrous Sodium Sulfate': {'tons'},
 'Borax-Decahydrate': {'tons'},
 'Borax-Pentahydrate': {'tons'},
 'Boric Acid': {'tons'},
 'Soda Ash': {'tons'},
 'Sodium Bi-Carbonate': {'tons'},
 'Phosphate Raw Ore': {'tons'},
 'Gold': {'oz'},
 'Copper Concentrate': {'tons'},
 'Lead Concentrate': {'tons'},
 'Zinc Concentrate': {'tons'},
 'Leonardite': {'tons'},
 'Langbeinite': {'tons'},
 'Muriate Of Potash-Granular': {'tons'},
 'Muriate Of Potash-Standard': {'tons'},
 'Potash': {'tons'},
 'Sylvite-Raw Ore': {'tons'},
 'Brine Barrels': {'bbl'},
 'Clay': {'tons'},
 'Magnesium Chloride Brine': {'tons'},
 'Manure Salts': {'tons'},
 'Gilsonite': {'tons'},
 'Limestonse': {'tons'},
 'Borate Pr

## Check Header Function

In [20]:
def check_header(file):
    columns = file.columns
    uncheckedCols = set(columns)
    
    for i in range(len(header)):
        # Checks if Field in file and in correct column
        if columns.contains(header[i]):
            if columns[i] == header[i]:
                print(header[i] + ": True")
            else:
                print(header[i] + ": Wrong order")
            uncheckedCols.remove(header[i])
        else:
            # Field not present in the file
            print(header[i] + ': N/A')
    # Prints all fields not in the format
    print('\nNew Cols:', uncheckedCols)
        
check_header(dfCYPro)

Month: N/A
Calendar Year: Wrong order
Land Class: N/A
Land Category: N/A
Commodity: N/A
Volume: N/A

New Cols: {'Onshore/Offshore', 'Production Volume', 'Product', 'State/Offshore Region', 'CPS/Planning Area', 'FIPS Code', 'Category'}


## Two  Potential Unit Approaches
Compare two Dictionaries
* Slightly Harder to implement
* More modular
* Better for documentation purposes

OR Compare two Sets
* Easier to implement
* Not as flexible

In [86]:
def make_unit_set(file):
    units = set()
    for u in file['Commodity']:
        units.add(u)
    return units


def check_unit_sets(file, default):
    file_units = make_unit_set(file)
    return file_units - default


default_set = make_unit_set(df_OLD)
# print('Unkown Commodity/Unit:', check_unit_sets(df_NEW, default_set))


def check_unit_dict(file, default):
    file_units = make_unit_dict(file)
    for k,v in file_units.items():
        if default.__contains__(k):
            print(default.get(k),' = ', v, '?')
            print(default.get(k))
            print('Unknown Unit:',v,'for item:',k)
        else:
            print('Unknown Item:',k)

write_units(make_unit_dict(dfCYPro))
default_dict = read_uconfig()
check_unit_dict(dfCYPro,default_dict)

{'mcf'}  =  {'mcf'} ?
Unknown Unit: {'mcf'} for item: Gas
{'bbl'}  =  {'bbl'} ?
Unknown Unit: {'bbl'} for item: Oil
{'tons'}  =  {'tons'} ?
Unknown Unit: {'tons'} for item: Coal
{'spcmn'}  =  {'spcmn'} ?
Unknown Unit: {'spcmn'} for item: Wavellite
{'tickets/pounds'}  =  {'tickets/pounds'} ?
Unknown Unit: {'tickets/pounds'} for item: Quartz Crystal
{'tons'}  =  {'tons'} ?
Unknown Unit: {'tons'} for item: Salt
{'Other', 'Thousands of Pounds', 'Kilowatt Hours'}  =  {'Other', 'Thousands of Pounds', 'Kilowatt Hours'} ?
Unknown Unit: {'Other', 'Thousands of Pounds', 'Kilowatt Hours'} for item: Geothermal - Electrical Generation
{'Hundreds of Gallons', 'Millions of BTUs'}  =  {'Hundreds of Gallons', 'Millions of BTUs'} ?
Unknown Unit: {'Hundreds of Gallons', 'Millions of BTUs'} for item: Geothermal - Direct Utilization
{'tons'}  =  {'tons'} ?
Unknown Unit: {'tons'} for item: Anhydrous Sodium Sulfate
{'tons'}  =  {'tons'} ?
Unknown Unit: {'tons'} for item: Borax-Decahydrate
{'tons'}  =  {'tons

In [46]:
help(dict)

Help on class dict in module builtins:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(mapping) -> new dictionary initialized from a mapping object's
 |      (key, value) pairs
 |  dict(iterable) -> new dictionary initialized as if via:
 |      d = {}
 |      for k, v in iterable:
 |          d[k] = v
 |  dict(**kwargs) -> new dictionary initialized with the name=value pairs
 |      in the keyword argument list.  For example:  dict(one=1, two=2)
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key, /)
 |      True if the dictionary has the specified key, else False.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __init__(self,